In [77]:
#May 28 - for ease for getting data out
import numpy as np
import matplotlib.pyplot as pl
import os

In [258]:
def getData(text):
    #get center z
    data = np.loadtxt(text, skiprows=0, max_rows=2, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)
    z = (float(data[1][1]) - float(data[1][0]))/2 + float(data[1][0])
    zstr = str(round(z, 5))
#     print(z)

    #check if errors are linear
    data = np.loadtxt(text, skiprows=2, max_rows=1, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)
#     print(data)

    linear = False
    if data[1] == 'linear':
        linear = True

    data = np.loadtxt(text, skiprows=0)

    data = np.transpose(data)

    masses = data[0]
    phi = data[1]
    err_up = data[2]
    err_down = data[3]

    errs = []

    if linear:
#         print("converting")

        for i in range(len(err_up)):
            errs.append((np.log10(err_up[i]), np.log10(err_down[i])))
            phi[i] = np.log10(data[1][i]) #need to convert phi values too

    else:    
        for i in range(len(err_up)):
            errs.append((err_up[i], err_down[i]))

    # print(errs)
    #check if the masses are evenly spaced
    even = True
    if len(masses) > 2:
        i = 1
        massDif = (masses[1] - masses[0])
        while i < (len(masses)-1) and even:
            if massDif != round((masses[i+1] - masses[i]), 14):
                even = False       
    else:
        even = False
        
    if not even:
        masses = np.array([10**m for m in masses])
        massesStr = print(masses)
        
        formatThese = [massesStr, str(phi.tolist()), str(errs)]
    else:
        formatThese = [str(masses.tolist()), str(phi.tolist()), str(errs)]
    print(formatThese)
        
    #space data nicely
    Strings = ["", "", ""]
    for i, value in enumerate(formatThese):
        
        while len(value) > 119 :
            split = 120
            ending = False
            
            if value[split] == ',':
#                 print("comma!")
                noComma = False
            else:
                noComma = True
            
            while noComma & (not ending):
#                 print(split)
                split +=1
#                 print(split, value[split])
                if value[split] == ',':
#                     print("comma!")
                    noComma = False
                if len(value[split:].split(',')) == 1:
                    ending = True

            if not ending:
                split +=1 #go to the next space

                Strings[i] = Strings[i] + value[:split] + "\n          "
                value = value[split:]
            else:
#                 print("ending:")
#                 print(value)
                Strings[i] = Strings[i] + value
                value = ""
        else:
#             print("short")
            Strings[i] = Strings[i] + value


#         print(Strings[i])
#         print()
    
    #writting the data to a string
    if even:
        incr = masses[1] - masses[0]
        minM = masses[0]
        maxM = masses[-1] + incr
        
        pyFile = " {}: {{'M': list(10**np.arange({}, {:.3f}, {:.3f})),\n    'phi': {},\n    'err': {}\n   }},".format(zstr, \
                                                    minM, maxM-incr, incr, Strings[1], Strings[2])
#         print(len(np.arange(minM, maxM-incr, incr)))

    else:
        pyFile = " {}: {{'M': {},\n    'phi': {},\n    'err': {}\n   }},".format(zstr,\
                                                    Strings[0], Strings[1], Strings[2])
#         print(len(masses))

    

#     print(len(phi), len(errs))
    
    return pyFile, z

In [259]:
directory = "../behroozi-2013-data-compilation/smf_ms"
names = ["a"]
zArr = []
for filename in os.listdir(directory):
    if filename.endswith(".smf"): 
        
        name = os.path.join(directory, filename)
        author = filename.split('_')[0]
        if author != names[-1]:
            print(zArr)
            print(f"\n{author}\n")
            zArr = []
            
        names.append(author)
        data, z = getData(name)
        print(data)   
        zArr.append(round(z, 5))
        
    else:
        continue

[]

bouwens

[3.08655320e+09 1.50201326e+09 7.30926599e+08 3.55691729e+08
 1.73090713e+08 8.42313510e+07 4.09896081e+07]
[None, '[-3.62838893005031, -3.75332766665861, -3.18905623622005, -2.77560406136976, -2.54695160369846, -2.41090498583641, -2.13527956994652]', '[(0.2, 0.2), (0.221848749616356, 0.221848749616356), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2)]']


TypeError: object of type 'NoneType' has no len()

In [242]:
file = "../behroozi-2013-data-compilation/smf_ms/moustakas_z0.575.smf"

data, z = getData(file)
# print(data)
        

IndexError: string index out of range

In [211]:
text = "1e,"
print(text.split(","))



['1e', '']


In [244]:
10**[1, 2, 3]

TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'list'